<a href="https://colab.research.google.com/github/mushhub/my-first-blockchain/blob/main/EVM_Solana_Bridge_onEVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.20;

import "@openzeppelin/contracts/token/ERC20/IERC20.sol";
import "@openzeppelin/contracts/access/Ownable.sol";
import "@openzeppelin/contracts/security/ReentrancyGuard.sol";

/**
 * @title EVMToSolanaBridge
 * @dev Contract to bridge tokens from EVM chains to Solana
 */
contract EVMToSolanaBridge is Ownable, ReentrancyGuard {
    // Events
    event BridgeInitiated(address indexed from, string toSolanaAddress, address token, uint256 amount, uint256 nonce);
    event BridgeCompleted(address indexed token, string toSolanaAddress, uint256 amount, uint256 nonce);
    event TokenAdded(address indexed token, bool status);
    event FeeUpdated(uint256 newFee);

    // Structs
    struct BridgeRequest {
        address from;
        string toSolanaAddress;
        address token;
        uint256 amount;
        uint256 timestamp;
        bool processed;
    }

    // State variables
    mapping(address => bool) public supportedTokens;
    mapping(uint256 => BridgeRequest) public bridgeRequests;
    uint256 public nonce;
    uint256 public bridgeFee;
    address public feeCollector;

    // Oracle address that will process the bridge requests
    address public oracle;

    // Modifier to restrict oracle functions
    modifier onlyOracle() {
        require(msg.sender == oracle, "Caller is not the oracle");
        _;
    }

    constructor(uint256 _bridgeFee, address _feeCollector) Ownable(msg.sender) {
        bridgeFee = _bridgeFee;
        feeCollector = _feeCollector;
        oracle = msg.sender; // Initially set oracle as the deployer
    }

    /**
     * @dev Add or remove a token from the supported list
     * @param _token Address of the token
     * @param _status Support status to set
     */
    function setTokenSupport(address _token, bool _status) external onlyOwner {
        supportedTokens[_token] = _status;
        emit TokenAdded(_token, _status);
    }

    /**
     * @dev Update bridge fee
     * @param _newFee New fee amount
     */
    function updateBridgeFee(uint256 _newFee) external onlyOwner {
        bridgeFee = _newFee;
        emit FeeUpdated(_newFee);
    }

    /**
     * @dev Set oracle address
     * @param _oracle New oracle address
     */
    function setOracle(address _oracle) external onlyOwner {
        require(_oracle != address(0), "Invalid oracle address");
        oracle = _oracle;
    }

    /**
     * @dev Set fee collector address
     * @param _feeCollector New fee collector address
     */
    function setFeeCollector(address _feeCollector) external onlyOwner {
        require(_feeCollector != address(0), "Invalid fee collector address");
        feeCollector = _feeCollector;
    }

    /**
     * @dev Initiate bridging of tokens from EVM to Solana
     * @param _token Address of the token to bridge
     * @param _amount Amount of tokens to bridge
     * @param _toSolanaAddress Destination Solana address
     */
    function bridgeToSolana(address _token, uint256 _amount, string calldata _toSolanaAddress) external payable nonReentrant {
        require(supportedTokens[_token], "Token not supported");
        require(_amount > 0, "Amount must be greater than 0");
        require(bytes(_toSolanaAddress).length > 0, "Invalid Solana address");
        require(msg.value >= bridgeFee, "Insufficient fee");

        // Transfer tokens from sender to this contract
        IERC20 token = IERC20(_token);
        require(token.transferFrom(msg.sender, address(this), _amount), "Token transfer failed");

        // Store bridge request
        uint256 currentNonce = nonce++;
        bridgeRequests[currentNonce] = BridgeRequest({
            from: msg.sender,
            toSolanaAddress: _toSolanaAddress,
            token: _token,
            amount: _amount,
            timestamp: block.timestamp,
            processed: false
        });

        // Transfer fee to fee collector
        (bool feeTransferSuccess, ) = feeCollector.call{value: msg.value}("");
        require(feeTransferSuccess, "Fee transfer failed");

        emit BridgeInitiated(msg.sender, _toSolanaAddress, _token, _amount, currentNonce);
    }

    /**
     * @dev Complete a bridge request (only called by the oracle)
     * @param _nonce Nonce of the bridge request to complete
     */
    function completeBridge(uint256 _nonce) external onlyOracle {
        BridgeRequest storage request = bridgeRequests[_nonce];
        require(!request.processed, "Request already processed");
        require(request.timestamp > 0, "Request does not exist");

        request.processed = true;

        emit BridgeCompleted(request.token, request.toSolanaAddress, request.amount, _nonce);
    }

    /**
     * @dev Withdraw tokens in case of emergency (only called by the owner)
     * @param _token Address of the token to withdraw
     * @param _recipient Address of the recipient
     */
    function emergencyWithdraw(address _token, address _recipient) external onlyOwner {
        require(_recipient != address(0), "Invalid recipient");

        if (_token == address(0)) {
            // Withdraw ETH
            uint256 balance = address(this).balance;
            require(balance > 0, "No ETH to withdraw");

            (bool success, ) = _recipient.call{value: balance}("");
            require(success, "ETH transfer failed");
        } else {
            // Withdraw ERC20 tokens
            IERC20 token = IERC20(_token);
            uint256 balance = token.balanceOf(address(this));
            require(balance > 0, "No tokens to withdraw");

            require(token.transfer(_recipient, balance), "Token transfer failed");
        }
    }

    /**
     * @dev Get pending bridge requests
     * @param _startNonce Starting nonce
     * @param _count Number of requests to fetch
     * @return array of pending requests
     */
    function getPendingRequests(uint256 _startNonce, uint256 _count) external view returns (BridgeRequest[] memory) {
        uint256 count = 0;

        // First count valid requests
        for (uint256 i = _startNonce; i < _startNonce + _count && i < nonce; i++) {
            if (!bridgeRequests[i].processed) {
                count++;
            }
        }

        // Then create the array and populate it
        BridgeRequest[] memory result = new BridgeRequest[](count);
        uint256 resultIndex = 0;

        for (uint256 i = _startNonce; i < _startNonce + _count && i < nonce; i++) {
            if (!bridgeRequests[i].processed) {
                result[resultIndex] = bridgeRequests[i];
                resultIndex++;
            }
        }

        return result;
    }
}